# 4. Exploratory Data Analysis
 
*Date: July 31, 2023*  
*Author: Alicia Larsen*     
*Institution: The Research Institute of Sweden (RISE)*   
*Contact: alicia.hh.larsen@gmail.com*   

This is the 5th notebook of 7, in the series "RISE Wildfire Prediction Using Machine Learning"

References: This notebook is based on the procedures in the notebook found on this [link](https://github.com/ornldaac/modis_restservice_qc_filter_Python/blob/master/modis_restservice_qc_filter_Python.ipynb). This notebook can also be found in /initial-eda/data-procurement/reference-notebook/download-modis-data-example-notebook.ipynb, on github.com:larsenalicia/RISE-wildfire-prediction.git

##### Keywords: LST, LSR, Fire, MODIS, Python

## Overview
This notebook will explore soma basic statistics of the datasets, show the feature values over time and a heatmap for one chosen date.

## Prerequisites: 

* Python 2 or 3   
* Libraries: requests, json, datetime, pandas, numpy, matplotlib
---

## Set-up
### Imports:

# 4. Exploratory Data Analysis
 
*Date: July 31, 2023*  
*Author: Alicia Larsen*     
*Institution: The Research Institute of Sweden (RISE)*   
*Contact: alicia.hh.larsen@gmail.com*   

This is the firest notebook of X, in the series "RISE Wildfire Prediction Using Machine Learning"

References: This notebook is based on the procedures in the notebook found on this [link](https://github.com/ornldaac/modis_restservice_qc_filter_Python/blob/master/modis_restservice_qc_filter_Python.ipynb). This notebook can also be found in /initial-eda/data-procurement/reference-notebook/download-modis-data-example-notebook.ipynb, on github.com:larsenalicia/RISE-wildfire-prediction.git

##### Keywords: LST, LSR, Fire, MODIS, Python

## Overview
This notebook will explore the access to 3 MODIS data products: 
* the Land Surface Temperature/Emissivity (MOD11A2), 
* Surface Reflectance (MOD09A1), and 
* Thermal Anomalies/Fire (MOD14A2).

## Prerequisites: 

* Python 2 or 3   
* Libraries: requests, json, datetime, pandas, numpy, matplotlib
---

## Set-up
### Imports:

In [ ]:
# Imports
import json
import datetime
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from globals.global_vars import url, header, coordinate_description, lat, lon, start_year, end_year, sensing_interval, products, bands
from procerdures.c_eda import stats, fire_stats

### Load required data:

In [ ]:
dataframes: dict = {}

# Iterate through the different frequences
for frequency in ['least', 'most']:

    # Iterate through the different filtering restrictions
    for restriction in ['hard', 'loose']:

        # Read a CSV in the right directory
        df_data = pd.read_csv(f'data/aggregation/raw/alldata_{frequency}_{restriction}_{start_year}-{end_year}_{coordinate_description}.csv')

        # Add the dataframe to a dictionary, for access
        dataframes[f'{frequency}_{restriction}'] = df_data.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'pixel'})

# Take a look at the keys
dataframes.keys()

Feel free to run this notebook several times with different dataframes. You can change the <code>dataframes[\<'key'>]</code>

In [ ]:
# Define the dataframe for this analysis
df_data = dataframes['most_loose']
df_data.head()

### Convert string index to type datetime

In [ ]:
# Check the type of the date-column
print(f'{df_data["date"][0]}: {type(df_data["date"][0])}')

In [ ]:
# Change the type of the date-column, and make a multi-index
df_data['date'] = pd.to_datetime(df_data['date'], format='%Y-%m-%d', errors='coerce')
df_data = df_data.set_index(['date', 'pixel'])

In [ ]:
# Check the type of the date-index
print(f'{df_data.index[0][0]}: {type(df_data.index[0][0])}')
df_data.head()

## Initial Statistics
### LST

In [ ]:
# LST statistics (Celsius)
stats(df_data, 'celsius')

### NDMI

In [ ]:
# NDMI statistics
stats(df_data, 'ndmi')


### EVI

In [ ]:
# NDMI statistics
stats(df_data, 'evi')


### Fire

In [ ]:
# Fire cout
fire_stats(df_data, 'fire')

In [ ]:
df_data.head()

In [ ]:
# More statistics regarding the fire dataset
df_pixel_fire = df_data[df_data['fire'] > 0].reset_index()['pixel'].to_frame()

print(f"""
FIRE PIXEL STATS
-------------------------------
fire occurance:             {len(df_pixel_fire.index)}
unique fire pixels:         {len(df_pixel_fire['pixel'].unique())}
middle number of pixels:    {df_data.reset_index()['pixel'].max()/2}
pixels below:               {round(len(df_pixel_fire[df_pixel_fire['pixel'] > df_data.reset_index()['pixel'].max()/2] / len(df_pixel_fire.index)), 3)*100}%
duplicate pixels:           {len(df_pixel_fire[df_pixel_fire.duplicated()])}
""")

## Predictors over time
### Preparation of dataframes

In [ ]:
# Separate the dataframes
df_temperature_c = df_data['celsius'].to_frame()
df_temperature_k = df_data['lst'].to_frame()
df_ndmi = df_data['ndmi'].to_frame()
df_evi = df_data['evi'].to_frame()
df_fire = df_data['fire'].to_frame()
df_temperature_c.head()

In [ ]:
# Aggregate the dataframes
df_temperature_c_stats = df_temperature_c['celsius'].groupby('date').agg(['mean', 'std', 'count'])
df_ndmi_stats = df_ndmi['ndmi'].groupby('date').agg(['mean', 'std', 'count'])
df_evi_stats = df_evi['evi'].groupby('date').agg(['mean', 'std', 'count'])

df_temperature_c_stats.head()

In [ ]:
# Dataframe for fire
df_fire_stats = df_fire['fire'].groupby('date').agg('mean').to_frame()
df_fire_stats.head()

### Plotting

In [ ]:
# LST
plt.rcParams['figure.figsize'] = (15,8)

fig, ax1 = plt.subplots()

ax1.set_xlabel('Date')
ax1.set_ylabel('LST (°C)')
ax1.plot(df_temperature_c_stats.index, df_temperature_c_stats['mean'], 'k-')
ax1.fill_between(df_temperature_c_stats.index, df_temperature_c_stats['mean']-df_temperature_c_stats['std'], df_temperature_c_stats['mean']+df_temperature_c_stats['std'])
ax1.tick_params(axis='y')
fig.autofmt_xdate()

ax2 = ax1.twinx()
ax2.set_ylabel('Valid Pixels')
ax2.bar(df_temperature_c_stats.index, df_temperature_c_stats['count'], 10, alpha = 0.3)
ax2.tick_params(axis='y')

plt.show()

In [ ]:
# NDMI
plt.rcParams['figure.figsize'] = (15,8)

fig, ax1 = plt.subplots()

ax1.set_xlabel('Date')
ax1.set_ylabel('NDMI')
ax1.plot(df_ndmi_stats.index, df_ndmi_stats['mean'], 'k-')
ax1.fill_between(df_ndmi_stats.index, df_ndmi_stats['mean']-df_ndmi_stats['std'], df_ndmi_stats['mean']+df_ndmi_stats['std'])
ax1.tick_params(axis='y')
fig.autofmt_xdate()

ax2 = ax1.twinx()
ax2.set_ylabel('Valid Pixels')
ax2.bar(df_ndmi_stats.index, df_ndmi_stats['count'], 10, alpha = 0.3)
ax2.tick_params(axis='y')

plt.show()

In [ ]:
# EVI
df_evi_stats1 = df_evi_stats.dropna()
plt.rcParams['figure.figsize'] = (15,8)

fig, ax1 = plt.subplots()

ax1.set_xlabel('Date')
ax1.set_ylabel('EVI')
ax1.plot(df_evi_stats1.index, df_evi_stats1['mean'], 'k-')
ax1.fill_between(df_evi_stats1.index, df_evi_stats1['mean']-df_evi_stats1['std'], df_evi_stats1['mean']+df_evi_stats1['std'])
ax1.tick_params(axis='y')
fig.autofmt_xdate()

ax2 = ax1.twinx()
ax2.set_ylabel('Valid Pixels')
ax2.bar(df_evi_stats1.index, df_evi_stats1['count'], 10, alpha = 0.3)
ax2.tick_params(axis='y')

plt.show()

In [ ]:
# Read in normalized data:
df_normalized = pd.read_csv(f'data/aggregation/normalized/alldata_most_loose_largest_{start_year}-{end_year}_{coordinate_description}.csv')

In [ ]:
# Check the type of the date-column
print(f'{df_normalized["date"][0]}: {type(df_normalized["date"][0])}')

In [ ]:
# Change the type of the date-column, and make a multi-index
df_normalized['date'] = pd.to_datetime(df_normalized['date'], format='%Y-%m-%d', errors='coerce')
df_normalized = df_normalized.set_index(['date', 'pixel'])

In [ ]:
# Check the type of the date-index
print(f'{df_normalized.index[0][0]}: {type(df_normalized.index[0][0])}')
df_normalized.head()

In [ ]:
# Re-format the df_normalized dataframe
df_normalized = df_normalized.reset_index().groupby('date').agg('mean')[['temperature_k', 'ndmi', 'evi']]
df_normalized.head()

In [ ]:
# Combined plot
plt.rcParams['figure.figsize'] = (15,8)

fig, ax1 = plt.subplots()

ax1.set_title('Fire-features over time', size=28, y=1.02)

ax1.set_xlabel('Time [date]', size=18, weight='bold')
ax1.set_ylabel('z-score', size=18, weight='bold')
lst = ax1.plot(df_normalized.index, df_normalized['temperature_k'], linewidth=3)
ndmi = ax1.plot(df_normalized.index, df_normalized['ndmi'], linewidth=1)
evi = ax1.plot(df_normalized.index, df_normalized['evi'], linewidth=3)

ax1.tick_params(axis='y')
fig.autofmt_xdate()

ax1.legend(['LST', 'NDMI', 'EVI'], fontsize=17)

plt.show()

In [ ]:
df_fire_stats['fire'].unique()

In [ ]:
# Fire
sns.set()
plt.rcParams['figure.figsize'] = (15,8)

fig, ax1 = plt.subplots()

ax1.set_title('Fire-pixel percentage over time', size=30)
ax1.set_xlabel('Time [date]', size=20, weight='bold')
ax1.set_ylabel('Fire pixels [%]', size=20, weight='bold')

ax1.plot(df_fire_stats.index, df_fire_stats['fire'], linewidth=3)
ax1.tick_params(axis='y')
fig.autofmt_xdate()

plt.show()

## Find date of fire

In [ ]:
# All fire-days
df_fire_stats[df_fire_stats['fire'] > 0]

In [ ]:
# The fire-date with the most fire
df_fire_stats[df_fire_stats['fire'] > 0].idxmax(axis='rows')

In [ ]:
# Manually define the date of interest
interest_date = '2018-07-13'
interest_datetime = datetime.date(2018, 7, 13)

We can see that 2018-07-13 had the largest fire, therefore, we will continue the exploratory analysis on this date.

## Heatmap
### LST

In [ ]:
# Define the relevant dataframe
df_lst_date = df_temperature_k.loc[interest_date, 'lst'].to_frame()
df_lst_date

In [ ]:
dimension = 201

# Create dataframe in matrix format
df_lst_matrix = pd.DataFrame()
# columns=[col for col in range(0, dimension)]

# For every pixel
for pix in df_lst_date.index:
    row = int(np.floor(pix / dimension))
    col = pix % dimension
    df_lst_matrix = df_lst_matrix
    df_lst_matrix.loc[row, col] = df_lst_date.loc[pix, 'lst']

In [ ]:
# Heatmap over LST
fig, ax = plt.subplots(figsize=(15,12))
sns.heatmap(data=df_lst_matrix, cmap="inferno", ax=ax)

ax.set_title('Satellite imagery of north of Sweden (201x201 km)', size=22, y=1.05)
ax.xaxis.tick_top()
labels = [i*4 for i in range(0,51)]
ax.set_xticklabels(labels, minor=False, rotation=90)

ax.set_xlabel('pixel x-coordinate [1km wide]', size=20, weight='bold')
ax.set_ylabel('pixel y-coordinate [1km wide]', size=20, weight='bold');

### NDMI

In [ ]:
# Define the relevand dataframe
df_ndmi_date = df_ndmi.loc[interest_date, 'ndmi'].to_frame()
df_ndmi_date

In [ ]:
dimension = 201

# Create dataframe in matrix format
df_ndmi_matrix = pd.DataFrame()

# For every pixel
for pix in df_ndmi_date.index:
    row = int(np.floor(pix / dimension))
    col = pix % dimension
    df_ndmi_matrix = df_ndmi_matrix
    df_ndmi_matrix.loc[row, col] = df_ndmi_date.loc[pix, 'ndmi']

In [ ]:
# Heatmap of LSR
fig, ax = plt.subplots(figsize=(15,12))
sns.heatmap(data=df_ndmi_matrix, cmap="inferno", ax=ax);

### EVI

In [ ]:
# Define relevant dataframe
df_evi_date = df_evi.loc[interest_date, 'evi'].to_frame()
df_evi_date.head(3)

In [ ]:
dimension = 201

# Create dataframe in matrix format
df_evi_matrix = pd.DataFrame()

# For every pixel
for pix in df_evi_date.index:
    row = int(np.floor(pix / dimension))
    col = pix % dimension

    df_evi_matrix = df_evi_matrix
    df_evi_matrix.loc[row, col] = df_evi_date.loc[pix, 'evi']

In [ ]:
# Heatmap of EVI
fig, ax = plt.subplots(figsize=(15,12))
sns.heatmap(data=df_evi_matrix, cmap="inferno", ax=ax);

### Fire

In [ ]:
# Define relevant dataframe
df_fire_date = df_fire.loc[interest_date, 'fire'].to_frame()
df_fire_date.head(3)

In [ ]:
# Check expected number of nan-values
df_fire.loc[interest_date,'fire'].isna().sum()

In [ ]:
dimension = 201

# Create dataframe in matrix format
df_fire_matrix = pd.DataFrame()

# For every pixel
for pix in df_fire_date.index:
    row = int(np.floor(pix / dimension))
    col = pix % dimension

    df_fire_matrix = df_fire_matrix
    df_fire_matrix.loc[row, col] = df_fire_date.loc[pix, 'fire']

In [ ]:
# Heatmap of fire
from matplotlib.patches import Patch

fig, ax = plt.subplots(figsize=(15,12))
sns.heatmap(data=df_fire_matrix, cmap=mpl.colors.ListedColormap(['#6fa8dcff', '#a64d79ff']), ax=ax, cbar=False)

ax.set_title('A spatial representation of fire pixels (2018-07-13)', size=24, y=1.05)
ax.xaxis.tick_top()
labels = [i*3 for i in range(0,67)]
ax.set_xticklabels(labels, minor=False, rotation=90)
legend_handles = [Patch(color="#6fa8dcff", label='Not Fire-pixel'),         # Blue
                    Patch(color="#a64d79ff", label='Fire-pixel'),
                    Patch(color='white', label='No Data')]           # Purple
ax.legend(handles=legend_handles, ncol=1, bbox_to_anchor=[1, 0], loc='lower right', fontsize=14, handlelength=.8)

ax.set_xlabel('pixel x-coordinate [1km wide]', size=20, weight='bold')
ax.set_ylabel('pixel y-coordinate [1km wide]', size=20, weight='bold');


## Wrap-up
Now you should have some basic understanding of the datasets.

Have a nice day!

/ Alicia